# 基本设置

In [44]:

import sys
import re
import time
import string

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor_1
import os
from sqlalchemy import create_engine
from pandas.io import sql
import re 
import warnings
warnings.filterwarnings('ignore')

# 读取文件

In [2]:
#这两个文件是自行爬取的正文
file_list1 = ['20180723_revise_clearn.xlsx','20180725_clearn.xlsx']

In [3]:
cir_data_raw_1 = pd.DataFrame()
for file_name in file_list1:
    tmp_data = pd.read_excel('data/result/{0}'.format(file_name))
    print(tmp_data.shape, file_name)
    cir_data_raw_1 = pd.concat([cir_data_raw_1, tmp_data], axis = 0)

# result.to_excel('data/result/class_data_raw_1.xlsx', index = False)
print('总数：', cir_data_raw_1.shape)
print(cir_data_raw_1[cir_data_raw_1['title']=='听说出来了一款保证续保的医疗险?真相如何?'])
cir_data_raw_1.head()

(1410, 5) 20180723_revise_clearn.xlsx
(1620, 5) 20180725_clearn.xlsx
总数： (3030, 5)
     label                  title  \
1554    噪音  听说出来了一款保证续保的医疗险?真相如何?   

                                              content  \
1554   听说出来了一款保证续保的医疗险?真相如何?\n\r2018-01-03 23:46\n"}}   

                                                    url              title_url  
1554  http://www.orz520.com/a/economy/2018/0103/8582...  听说出来了一款保证续保的医疗险?真相如何?  


,label,title,content,url,title_url
0,噪音,做好规划不当月光族不做守财奴,一一妈妈说理财·出品\n (图片)\n\r月光族困惑：···这个月才过半，各种经费不够啊！...,https://xueqiu.com/1983578372/98551727,妈妈说理财: 做好规划不当月光族不做守财奴 一一妈妈说理财·出品 月光族困惑：···这个月才...
1,噪音,做车险代理的前景如何 谊信合让你的财富事业迈上新高度,小本创业网\n\r»\n\r加盟项目资讯›\n\r»\n\r资讯中心\n\r»\n\r做车...,http://jxhb.91cy.cn/119134.shtml,做车险代理的前景如何 谊信合让你的财富事业迈上新高度
2,产品销售,坐飞机必须买保险？航意险适合哪些人？,坐飞机必须买保险？航意险适合哪些人？\n\r 随着经济的发展，大家对生活品质的追求，越来...,http://www.dtnews.cn/2018/january/99502C93.html,坐飞机必须买保险？航意险适合哪些人？\r\n - 大同新闻网
3,噪音,左权县人民政府金融工作办公室2017年统筹整合财政资金项目公示,\n\r为推进我县\n“五位一体”扶贫小额信贷工作，各承贷银行对全县建档立卡贫困户累计备案...,http://www.zqxzf.com/newscont.asp?id=61811&pid...,左权县人民政府金融工作办公室2017年统筹整合财政资金项目公示
4,消费服务,昨天车险报案比平常翻倍,(图片)\n\r雪中骑行滑倒 朱小军 摄\n\r 一场暴雪让扬州各家保险公司忙翻了天，车...,http://3g.k.sohu.com/t/n253854349,昨天车险报案比平常翻倍


In [4]:
cir_data_raw_1['label'].value_counts()

噪音        663
监管        649
公司内部管理    434
其他相关报道    425
资本市场      302
产品销售      255
消费服务      157
行业        145
Name: label, dtype: int64

In [28]:
#这些文件是自身带有正文的文件
file_list2 = ['20180727_circ_class_1.xlsx', '20180803_circ_class_1.xlsx', 
              '20180803_circ_class_2.xlsx', '20180803_circ_class_3.xlsx'] # 多个 sheet

In [29]:
import xlrd

In [31]:
cir_data_raw_2 = pd.DataFrame()
for file_name in file_list2:
    currentPath  = 'data/raw_data/{0}'.format(file_name)
    excel = xlrd.open_workbook(currentPath)
    sheet_names = [sheet.name for sheet in excel.sheets()]
    for sheet_name in sheet_names:    
        tmp_data = pd.read_excel(currentPath, sheet_name)
        if 'label' in tmp_data.columns:
            tmp_data = tmp_data[['label', 'title', 'content']]
            print('去空值前： ', tmp_data.shape, file_name, sheet_name)
            tmp_data = tmp_data.dropna(subset = ['title'], axis = 0)
            print('去空值后： ', tmp_data.shape, file_name, sheet_name)
            cir_data_raw_2 = pd.concat([cir_data_raw_2, tmp_data], axis = 0)

cir_data_raw_2['label'] = cir_data_raw_2['label'].apply(lambda x:x.strip())    
print('总数：', cir_data_raw_2.shape)
cir_data_raw_2.head()

去空值前：  (4478, 3) 20180727_circ_class_1.xlsx 数据
去空值后：  (4477, 3) 20180727_circ_class_1.xlsx 数据
去空值前：  (74, 3) 20180803_circ_class_1.xlsx Sheet1
去空值后：  (74, 3) 20180803_circ_class_1.xlsx Sheet1
去空值前：  (706, 3) 20180803_circ_class_1.xlsx Sheet2
去空值后：  (706, 3) 20180803_circ_class_1.xlsx Sheet2
去空值前：  (397, 3) 20180803_circ_class_1.xlsx Sheet3
去空值后：  (397, 3) 20180803_circ_class_1.xlsx Sheet3
去空值前：  (519, 3) 20180803_circ_class_1.xlsx Sheet4
去空值后：  (519, 3) 20180803_circ_class_1.xlsx Sheet4
去空值前：  (117, 3) 20180803_circ_class_2.xlsx Sheet1
去空值后：  (117, 3) 20180803_circ_class_2.xlsx Sheet1
去空值前：  (73, 3) 20180803_circ_class_2.xlsx Sheet2
去空值后：  (73, 3) 20180803_circ_class_2.xlsx Sheet2
去空值前：  (113, 3) 20180803_circ_class_2.xlsx Sheet3
去空值后：  (113, 3) 20180803_circ_class_2.xlsx Sheet3
去空值前：  (94, 3) 20180803_circ_class_2.xlsx Sheet4
去空值后：  (94, 3) 20180803_circ_class_2.xlsx Sheet4
去空值前：  (49, 3) 20180803_circ_class_2.xlsx Sheet5
去空值后：  (49, 3) 20180803_circ_class_2.xlsx Sheet5
去空值前：  (216, 3

,label,title,content
0,产品销售,做人干干净净，善良终有好报！,牵手国寿 中国人寿，一言九鼎；相知多年，值得托付！ 做人，就两个字：善良。 善良的人，永远都...
1,产品销售,做个好人心正身安魂梦稳 行些善事天知地鉴鬼神钦 以人为本 以天为则,完整财富=家庭 健康 教育 事业 服务 金钱 精神 传播聚财 守财 传财 散财 善财之道 家...
2,产品销售,昨天这对夫妻的聊天记录曝光了，网友们都炸了，女人，要不娘家有钱，要不自己有保险...,爱华云服务 让生活更美好，让华夏充满爱！ 关注 什么是女人的安全感？不是什么漂亮的...
3,其他相关报道,尊享代驾 安全快捷送您回家,为回馈广大客户，富德生命人寿第十二届客服节期间特意为客户提供了免费补发保单、尊享代驾等增值服...
4,产品销售,尊享e生旗舰版震撼升级！上海质子重离子医院治疗也可100%报销！！！,各位亲爱的会员，国民医保再次升级，现在购买尊享e生旗舰版，可自主增加质子重离子医院治...


In [32]:
cir_data_raw_2['label'].value_counts()

噪音        2713
消费服务      1416
其他相关报道    1031
监管         839
行业         618
公司内部管理     467
资本市场       281
产品销售       135
Name: label, dtype: int64

In [33]:
cir_data_raw = pd.concat([cir_data_raw_1[['title', 'content', 'label']], 
                          cir_data_raw_2[['title', 'content', 'label']]], 
                         axis = 0)
cir_data_raw['title_content'] = cir_data_raw['title'].astype(str) + '。' + cir_data_raw['content'].astype(str)
cir_data_raw['label'] = cir_data_raw['label'].apply(lambda x:x.strip())
cir_data_raw['index'] = range(cir_data_raw.shape[0])
print(cir_data_raw.shape)
cir_data_raw.head()

(10530, 5)


,title,content,label,title_content,index
0,做好规划不当月光族不做守财奴,一一妈妈说理财·出品\n (图片)\n\r月光族困惑：···这个月才过半，各种经费不够啊！...,噪音,做好规划不当月光族不做守财奴。 一一妈妈说理财·出品\n (图片)\n\r月光族困惑：···...,0
1,做车险代理的前景如何 谊信合让你的财富事业迈上新高度,小本创业网\n\r»\n\r加盟项目资讯›\n\r»\n\r资讯中心\n\r»\n\r做车...,噪音,做车险代理的前景如何 谊信合让你的财富事业迈上新高度。 小本创业网\n\r»\n\r加盟项目...,1
2,坐飞机必须买保险？航意险适合哪些人？,坐飞机必须买保险？航意险适合哪些人？\n\r 随着经济的发展，大家对生活品质的追求，越来...,产品销售,坐飞机必须买保险？航意险适合哪些人？。 坐飞机必须买保险？航意险适合哪些人？\n\r 随着...,2
3,左权县人民政府金融工作办公室2017年统筹整合财政资金项目公示,\n\r为推进我县\n“五位一体”扶贫小额信贷工作，各承贷银行对全县建档立卡贫困户累计备案...,噪音,左权县人民政府金融工作办公室2017年统筹整合财政资金项目公示。 \n\r为推进我县\n“五...,3
4,昨天车险报案比平常翻倍,(图片)\n\r雪中骑行滑倒 朱小军 摄\n\r 一场暴雪让扬州各家保险公司忙翻了天，车...,消费服务,昨天车险报案比平常翻倍。 (图片)\n\r雪中骑行滑倒 朱小军 摄\n\r 一场暴雪让扬州...,4


In [34]:
cir_data_raw.groupby('label')['title'].count()

label
产品销售       390
公司内部管理     901
其他相关报道    1456
噪音        3376
消费服务      1573
监管        1488
行业         763
资本市场       583
Name: title, dtype: int64

In [35]:
test_index = []
val_index = []
train_index = []

for label in cir_data_raw['label'].unique():
    indexs = cir_data_raw[cir_data_raw['label'] == label]['index'].tolist()
    test_index += indexs[:40] # 40
    val_index += indexs[40:60] # 20
    train_index += indexs[60:]

In [36]:
coprus_save_filename = 'data/cnews.test.txt'

test_data = np.array(cir_data_raw[cir_data_raw['index'].isin(test_index)][['label', 'title_content']])
print(test_data.shape)

f = open(coprus_save_filename, "w+", encoding='UTF-8')
for [l, d] in test_data:
    f.write(l + '----' + d + '\n')
f.close()

(320, 2)


In [37]:
coprus_save_filename = 'data/cnews.val.txt'

val_data = np.array(cir_data_raw[cir_data_raw['index'].isin(val_index)][['label', 'title_content']])
print(val_data.shape)

f = open(coprus_save_filename, "w+", encoding='UTF-8')
for [l, d] in val_data:
    f.write(l + '----' + d + '\n')
f.close()

(160, 2)


In [38]:
coprus_save_filename = 'data/cnews.train.txt'

train_data = np.array(cir_data_raw[cir_data_raw['index'].isin(train_index)][['label', 'title_content']])
print(train_data.shape)

f = open(coprus_save_filename, "w+", encoding='UTF-8')
for [l, d] in train_data:
    f.write(l + '----' + d + '\n')
f.close()

(10050, 2)


In [39]:
label_dic={'监管':1,'行业':2,'产品销售':3,'资本市场':4,'公司内部管理':5,'消费服务':6,'其他相关报道':7,'噪音':8}
label_dic.keys()

dict_keys(['监管', '行业', '产品销售', '资本市场', '公司内部管理', '消费服务', '其他相关报道', '噪音'])

# 预处理

In [40]:
titles = pre_cor_1.handle_contents(cir_data_raw['title'].tolist())
print(len(titles))

save_filename = 'data/titles.txt'
fid = open(save_filename, "w+", encoding='UTF-8')
for data in titles:
    fid.write(data + '\n')
fid.close()

10530


In [41]:
contents = pre_cor_1.handle_contents(cir_data_raw['content'].tolist())
print(len(contents))
print(contents[:2])
contents = [re.sub(r'[a-z]*', '', x) for x in contents]
print(len(contents))
print(contents[:2])

coprus_save_filename = 'data/contents.txt'
f = open(coprus_save_filename, "w+", encoding='UTF-8')
for data in contents:
    f.write(data + '\n')
f.close()

10530
['一一 妈妈 理财 出品 图片 月光族 困惑 过半 各种 经费 不够 小伙伴 同感 拯救 钱包 包养 求蹭饭 前段时间 一一 妈妈 台北 忙里偷闲 感受 台北 风景 同行 导游 介绍 城市 规划 说道 一个 城市 新市区 市区 最大 差别 新市区 通常 都市计划 然后 居民 进驻 市区 居民 后来 都市计划 例如 新北市 永和 地区 典型 市区 台北市 信义 区则 新市区 典范 看看 永和 每条 道路 狭窄 弯弯曲曲 计程车 司机 永和 没有 一条 建筑物 一栋 连著 一栋 中间 几乎 没有 空地 看看 信义 每条 道路 除了 非常 宽大 之外 非常 笔直 建筑物 空地 非常 充足 整个 地区 一点 不会 拥挤 感觉 地区 主要用途 商业中心 住宅区 特别 规划 一定 范围 规划 重要性 新旧 市区 只差 都市计划 时间 其实 凡事 必须 事先 规划 未来 照著 计划 方向 发展 结果 预期 一致 家庭 财务 一样 如果 没有 事先 规划 走入 两个 极端 不是 过度 花费 太过 节俭 因為 没有 规划 知道 每月 多少 过度 乐观 来说 有钱 不管 未来 如何 享受 容易 月光族 过度 保守 因為 未来 没有 把握 造成 过度 担忧 有钱 不敢 享用 之后 徒留 大笔 财富 子孙 贫穷 日子 注定 守财奴 月光族 守财奴 两种 极端 不是 所要 理想 境界 即时 享乐 能够 顾及 未来 需求 达到 目标 唯有 做好 财务 规划 机会 达成 财务 规划 基本上 分為 短期 长期 规划 两种 短期 规划 做好 一年 财务预算 长期 规划 建立 家庭 终身 财务 蓝图 图片 短期 财务 规划 短期 财务 规划 主要 针对 一年 收入 支出 做出 最好 调配 例如 预估 当年 多少 收入 费用 必备 支出 奢侈 开销 每月 应该 储蓄 多少 金额 才能 配合 长期 财务 规划 短期 规划 重点 年度 收支 做好 年度预算 重要 预备 当年 如何 开支 一般来说 年收入 薪资 相当 固定 但是 支出 变化 比较 究其原因 奢侈 费用 聚餐 国外 旅游 治装费 一不小心 超支 需要 严谨 控制 才行 现在 很多 好用 工具 一直 随手 提前 预算表 记录 日常 开销 每月 进行 及时 图表 分析 总结 图片 长期 财务 规划 长期 财务 规划 终身 财务 蓝图 

In [42]:
label = cir_data_raw['label'].tolist()
print(len(label))

coprus_save_filename = 'data/labels.txt'
f = open(coprus_save_filename, "w+", encoding='UTF-8')
for data in label:
    f.write(data + '\n')
f.close()

10530


In [43]:
cir_data_raw.to_excel('data/result/title_content_label.xlsx', index = False)
cir_data_raw.shape

(10530, 5)